# Recipe Recommendation System

This notebook demonstrates how the recipe recommendation system works. We'll go through the following steps:
1. Data Loading and Exploration
2. Feature Engineering
3. Model Building
4. Making Recommendations

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

## 1. Data Loading and Exploration

In [ ]:
# Load the dataset
data = pd.read_csv("recipe_final (1).csv")

# Display basic information about the dataset
print("Dataset Shape:", data.shape)
print("\nColumns in the dataset:")
print(data.columns.tolist())
print("\nSample data:")
data.head()

## 2. Feature Engineering

We'll process two types of features:
1. Numerical features (calories, fat, etc.)
2. Text features (ingredients)

In [ ]:
# Process ingredients using TF-IDF
vectorizer = TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(data['ingredients_list'])

# Display some statistics about the ingredients features
print("Number of unique ingredients features:", X_ingredients.shape[1])
print("Sample ingredients vocabulary:")
print(list(vectorizer.vocabulary_.items())[:10])

In [ ]:
# Normalize numerical features
numerical_features = ['calories', 'fat', 'carbohydrates', 'protein', 'cholesterol', 'sodium', 'fiber']
scaler = StandardScaler()
X_numerical = scaler.fit_transform(data[numerical_features])

# Display statistics of numerical features
print("Numerical features statistics:")
data[numerical_features].describe()

## 3. Model Building

We'll combine both numerical and text features and build a K-Nearest Neighbors model.

In [ ]:
# Combine features
X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

# Train KNN Model
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X_combined)

print("Model trained on feature matrix of shape:", X_combined.shape)

## 4. Making Recommendations

Let's create a function to get recipe recommendations based on input features.

In [ ]:
def recommend_recipes(input_features):
    """
    Get recipe recommendations based on input features.
    
    Parameters:
    input_features (list): List containing [calories, fat, carbs, protein, cholesterol, sodium, fiber, ingredients]
    
    Returns:
    DataFrame: Top 5 recommended recipes
    """
    input_features_scaled = scaler.transform([input_features[:7]])
    input_ingredients_transformed = vectorizer.transform([input_features[7]])
    input_combined = np.hstack([input_features_scaled, input_ingredients_transformed.toarray()])
    
    distances, indices = knn.kneighbors(input_combined)
    recommendations = data.iloc[indices[0]]
    return recommendations[['recipe_name', 'ingredients_list', 'image_url']].head(5)

### Example: Getting Recipe Recommendations

In [ ]:
# Example input features
sample_input = [
    300,  # calories
    10,   # fat
    40,   # carbohydrates
    15,   # protein
    20,   # cholesterol
    500,  # sodium
    5,    # fiber
    "chicken, rice, vegetables"  # ingredients
]

# Get recommendations
recommendations = recommend_recipes(sample_input)
print("Recommended Recipes:")
recommendations